In [1]:
import sys
sys.path.append('E:/xinlib')
import numpy as np
from xintab import DataLoader
import xintab

import mxnet as mx
from mxnet import gluon, autograd, nd
from mxnet.gluon import nn


class Xinet(nn.Block):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.net = nn.Sequential()
        with self.name_scope():
            self.net.add(
                nn.Conv2D(channels=20, kernel_size=5),
                ### 添加了批量归一化层
                nn.BatchNorm(axis=1),
                nn.Activation(activation='relu'),
                nn.MaxPool2D(pool_size=2, strides=2),
                # 第二层卷积
                nn.Conv2D(channels=50, kernel_size=3),
                nn.BatchNorm(axis=1),
                nn.Activation(activation='relu'),
                nn.MaxPool2D(pool_size=2, strides=2),
                # 第一层全连接
                nn.Dense(128, activation="relu"),
                nn.Dense(10, activation='relu')
            )
    def forward(self, x):
        return self.net(x)
            
ctx = xintab.try_gpu()
batch_size = 256

Using CNTK backend
C:\ProgramData\Anaconda3\lib\site-packages\keras\backend\cntk_backend.py:21: UserWarning: CNTK backend warning: GPU is not detected. CNTK's CPU version is not fully optimized,please run with GPU to get better performance.
  'CNTK backend warning: GPU is not detected. '


In [2]:
mnist = nn.Sequential()
mnist.add(Xinet(prefix='train_'))
mnist.initialize(ctx=ctx, init=mx.init.Xavier())
loader = DataLoader('cifar10')
loss = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(mnist.collect_params(), 'rmsprop', {'learning_rate': 0.1, 'gamma1': 0.9})
loader.train(mnist, loss, trainer, 10, batch_size, 0.75)
loader.myh5.close()

Start training on  gpu(0)
Epoch 0. Loss: 22.6518, Train acc 0.100209, valid acc 0.0961944, Time 7.7 sec
_____________________________________________________________________________________
Test acc 0.100509
Epoch 1. Loss: 2.37082, Train acc 0.100888, valid acc 0.0967026, Time 6.611 sec
_____________________________________________________________________________________
Epoch 2. Loss: 2.28054, Train acc 0.102045, valid acc 0.095579, Time 6.575 sec
_____________________________________________________________________________________
Epoch 3. Loss: 2.30259, Train acc 0.101478, valid acc 0.095579, Time 6.59798 sec
_____________________________________________________________________________________
Epoch 4. Loss: 2.30259, Train acc 0.10145, valid acc 0.095579, Time 6.57102 sec
_____________________________________________________________________________________
Epoch 5. Loss: 2.30259, Train acc 0.101337, valid acc 0.095579, Time 6.56302 sec
_______________________________________________